In [ ]:
#Task 3

In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

def solve_goal_programming(fix_vals=None, objective_priority='d1_minus'):

    G = LpVariable("G", lowBound=0)
    T1 = LpVariable("T1", lowBound=0)
    T2 = LpVariable("T2", lowBound=0)
    C = LpVariable("C", lowBound=0)

    d1_minus = LpVariable("d1_minus", lowBound=0)
    d1_plus = LpVariable("d1_plus", lowBound=0)
    d2_plus = LpVariable("d2_plus", lowBound=0)
    d3_plus = LpVariable("d3_plus", lowBound=0)
    d4_plus = LpVariable("d4_plus", lowBound=0)


    revenue = 1.5*G + 25*T1 + 15*T2
    budget_constr = revenue - (1000 - C) + d1_minus - d1_plus == 0

    cut_constr = C - 150 + d2_plus == 0

    upper_tax_constr = 0.5*G + 5*T1 + 15*T2 - 550 + d3_plus == 0

    lower_tax_constr = G + 20*T1 - 350 + d4_plus == 0

    rate_constr = T2 - T1 >= 0

    model = LpProblem("PreemptiveGoalProgramming", LpMinimize)

    model += budget_constr
    model += cut_constr
    model += upper_tax_constr
    model += lower_tax_constr
    model += rate_constr

    # If we have fixed values for higher priority deviations, fix them
    if fix_vals is not None:
        for var_name, val in fix_vals.items():
            if var_name == 'd1_minus':
                model += d1_minus == val
            if var_name == 'd1_plus':
                model += d1_plus == val
            if var_name == 'd2_plus':
                model += d2_plus == val
            if var_name == 'd3_plus':
                model += d3_plus == val
            if var_name == 'd4_plus':
                model += d4_plus == val

    if objective_priority == 'd1_minus':
        model.setObjective(d1_minus)
    elif objective_priority == 'd2_plus':
        model.setObjective(d2_plus)
    elif objective_priority == 'd3_plus':
        model.setObjective(d3_plus)
    elif objective_priority == 'd4_plus':
        model.setObjective(d4_plus)

    # Solve
    model.solve()

    sol = {
        'status': LpStatus[model.status],
        'G': value(G),
        'T1': value(T1),
        'T2': value(T2),
        'C': value(C),
        'd1_minus': value(d1_minus),
        'd1_plus': value(d1_plus),
        'd2_plus': value(d2_plus),
        'd3_plus': value(d3_plus),
        'd4_plus': value(d4_plus),
        'revenue': value(revenue)
    }
    return sol


sol1 = solve_goal_programming(objective_priority='d1_minus')

fix_vals = {'d1_minus': sol1['d1_minus'], 'd1_plus': sol1['d1_plus']}

sol2 = solve_goal_programming(fix_vals=fix_vals, objective_priority='d2_plus')

fix_vals['d2_plus'] = sol2['d2_plus']

sol3 = solve_goal_programming(fix_vals=fix_vals, objective_priority='d3_plus')

fix_vals['d3_plus'] = sol3['d3_plus']


sol4 = solve_goal_programming(fix_vals=fix_vals, objective_priority='d4_plus')


print("Final Preemptive GP Solution:")
print("Status:", sol4['status'])
print("G (gas tax):", sol4['G'], "cents/gal")
print("T1 (first $30k):", sol4['T1'])
print("T2 (over $30k):", sol4['T2'])
print("C (spending cut):", sol4['C'], "billion")
print("Deviations:", "d1_minus:", sol4['d1_minus'], "d1_plus:", sol4['d1_plus'], 
      "d2_plus:", sol4['d2_plus'], "d3_plus:", sol4['d3_plus'], "d4_plus:", sol4['d4_plus'])
print("Revenue:", sol4['revenue'], "billion")




Final Preemptive GP Solution:
Status: Optimal
G (gas tax): 0.0 cents/gal
T1 (first $30k): 15.0
T2 (over $30k): 31.666667
C (spending cut): 150.0 billion
Deviations: d1_minus: 0.0 d1_plus: 0.0 d2_plus: 0.0 d3_plus: 0.0 d4_plus: 50.0
Revenue: 850.000005 billion
